In [1]:
import pickle
import joblib
import time

import pandas as pd
import numpy as np

from numpy import mean, std
from datetime import timedelta
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, KBinsDiscretizer, PolynomialFeatures

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_27900/2924341845.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
path_to_data_folder =  '/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/process_mining_data'

In [3]:
# extract the 1-10 lags

df_2017 = pd.read_csv(f"{path_to_data_folder}/BPI_2017_naive.csv")

df_2017['concept:name - lag_1'] = df_2017['concept:name'].shift(1)
df_2017['concept:name - lag_2'] = df_2017['concept:name'].shift(2)
df_2017['concept:name - lag_3'] = df_2017['concept:name'].shift(3)

df_2017 = df_2017.loc[3:]

# define target
df_2017['concept:name next'] = df_2017['concept:name'].shift(-1)

df_2017 = df_2017.drop(df_2017.index[-1])

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_27900/2026526755.py:3: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2017 = pd.read_csv(f"{path_to_data_folder}/BPI_2017_naive.csv")


In [4]:
# only for testing 
# total_rows = df_2017.shape[0]

# # Calculate the index to divide the DataFrame 
# split_index = total_rows // 2

# # Split the DataFrame into two halves
# df_2017 = df_2017.iloc[:split_index, :]

df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,time_to_next_event,time_to_next_event_seconds,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,concept:name next
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,New credit,...,0 days 00:00:00.011000,0.01,35.17,0 days 00:01:21.088000,5 days 16:10:11.014999397,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,W_Complete application
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,New credit,...,0 days 00:00:00.010000,0.01,0.01,0 days 00:01:21.099000,5 days 08:22:12.629814592,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Concept
5,5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413,Existing loan takeover,New credit,...,4 days 23:34:08.079000,430448.08,179447.74,0 days 00:01:21.109000,5 days 05:51:35.263530324,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Create Application
6,6,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500,Home improvement,New credit,...,0 days 00:00:00.049000,0.05,0.04,0 days 00:00:00,4 days 18:38:55.010316100,A_Create Application,A_Concept,W_Complete application,W_Handle leads,A_Submitted
7,7,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549,Home improvement,New credit,...,0 days 00:00:00.191000,0.19,0.30,0 days 00:00:00.049000,4 days 18:50:11.306445745,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201084,1201084,statechange,User_99,A_Pending,Application,ApplState_1437687321,complete,2017-02-01 09:58:43.970,Remaining debt home,New credit,...,0 days 00:00:00.002000,0.00,0.00,75 days 14:18:05.736000,19 days 15:02:47.018194346,W_Call incomplete files,O_Accepted,W_Call incomplete files,A_Incomplete,W_Validate application
1201085,1201085,Deleted,User_99,W_Validate application,Workflow,Workitem_1690791617,ate_abort,2017-02-01 09:58:43.972,Remaining debt home,New credit,...,0 days 00:00:00.009000,0.01,0.02,75 days 14:18:05.738000,20 days 05:59:20.038348289,W_Call incomplete files,A_Pending,O_Accepted,W_Call incomplete files,O_Cancelled
1201086,1201086,statechange,User_99,O_Cancelled,Offer,OfferState_2080870299,complete,2017-02-01 09:58:43.981,Remaining debt home,New credit,...,0 days 00:00:00.007000,0.01,0.01,75 days 14:18:05.747000,20 days 07:06:36.925158626,W_Call incomplete files,W_Validate application,A_Pending,O_Accepted,O_Cancelled
1201087,1201087,statechange,User_99,O_Cancelled,Offer,OfferState_1140159487,complete,2017-02-01 09:58:43.988,Remaining debt home,New credit,...,0 days 00:00:00.006000,0.01,0.01,75 days 14:18:05.754000,20 days 04:04:09.814735216,W_Call incomplete files,O_Cancelled,W_Validate application,A_Pending,O_Cancelled


In [5]:
df_2017.columns

Index(['Unnamed: 0', 'Action', 'org:resource', 'concept:name', 'EventOrigin',
       'EventID', 'lifecycle:transition', 'time:timestamp', 'case:LoanGoal',
       'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID',
       'timestamp_difference', 'event_seq', 'time_to_next_event',
       'time_to_next_event_seconds', 'predicted_time_to_next_event_seconds',
       'elapsed_time_from_start', 'predicted_start_time', 'predicted_step',
       'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3',
       'concept:name next'],
      dtype='object')

In [6]:
# Prepare data
X = df_2017[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]
y = df_2017['concept:name next']

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
X['concept:name'] = label_encoder.fit_transform(X['concept:name'])
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the Random Forest Classifier
rf_clf = RandomForestClassifier(n_jobs=-1)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [21, 22, 23],
    'max_depth': [18, 19, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform nested cross-validation
outer_cv = KFold(n_splits=3, shuffle=False, random_state=None)  # 3-fold outer cross-validation
inner_cv = KFold(n_splits=5, shuffle=False, random_state=None)  # 5-fold inner cross-validation

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=inner_cv)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Print the results
print(f"Best params: {grid_search.best_params_}")
print(f'Nested Cross-Validation Accuracy: {grid_search.best_score_:.4f}')
print(best_model.score(X_test, y_test))


/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_27900/2884791935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_27900/2884791935.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_27900/2884791935.py:9: SettingW

Best params: {'max_depth': 19, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 22}
Nested Cross-Validation Accuracy: 0.8355
0.8353120915168721


In [7]:
df_2017['concept:name next rfc'] = label_encoder.inverse_transform(best_model.predict(X))

# decode labels

In [8]:
df_2017 = df_2017.drop(['concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3'], axis=1, errors='ignore')

In [9]:
df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,timestamp_difference,event_seq,time_to_next_event,time_to_next_event_seconds,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,concept:name next,concept:name next rfc
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,New credit,...,0 days 00:01:20.618000,4,0 days 00:00:00.011000,0.01,35.17,0 days 00:01:21.088000,5 days 16:10:11.014999397,W_Handle leads,W_Complete application,W_Complete application
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,New credit,...,0 days 00:00:00.011000,5,0 days 00:00:00.010000,0.01,0.01,0 days 00:01:21.099000,5 days 08:22:12.629814592,W_Complete application,A_Concept,A_Concept
5,5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413,Existing loan takeover,New credit,...,0 days 00:00:00.010000,6,4 days 23:34:08.079000,430448.08,179447.74,0 days 00:01:21.109000,5 days 05:51:35.263530324,A_Concept,A_Create Application,A_Create Application
6,6,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500,Home improvement,New credit,...,0 days 00:23:35.087000,1,0 days 00:00:00.049000,0.05,0.04,0 days 00:00:00,4 days 18:38:55.010316100,A_Create Application,A_Submitted,A_Submitted
7,7,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549,Home improvement,New credit,...,0 days 00:00:00.049000,2,0 days 00:00:00.191000,0.19,0.30,0 days 00:00:00.049000,4 days 18:50:11.306445745,A_Submitted,W_Handle leads,W_Handle leads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201084,1201084,statechange,User_99,A_Pending,Application,ApplState_1437687321,complete,2017-02-01 09:58:43.970,Remaining debt home,New credit,...,0 days 00:00:00.005000,72,0 days 00:00:00.002000,0.00,0.00,75 days 14:18:05.736000,19 days 15:02:47.018194346,W_Call incomplete files,W_Validate application,W_Validate application
1201085,1201085,Deleted,User_99,W_Validate application,Workflow,Workitem_1690791617,ate_abort,2017-02-01 09:58:43.972,Remaining debt home,New credit,...,0 days 00:00:00.002000,73,0 days 00:00:00.009000,0.01,0.02,75 days 14:18:05.738000,20 days 05:59:20.038348289,W_Call incomplete files,O_Cancelled,W_Validate application
1201086,1201086,statechange,User_99,O_Cancelled,Offer,OfferState_2080870299,complete,2017-02-01 09:58:43.981,Remaining debt home,New credit,...,0 days 00:00:00.009000,74,0 days 00:00:00.007000,0.01,0.01,75 days 14:18:05.747000,20 days 07:06:36.925158626,W_Call incomplete files,O_Cancelled,W_Validate application
1201087,1201087,statechange,User_99,O_Cancelled,Offer,OfferState_1140159487,complete,2017-02-01 09:58:43.988,Remaining debt home,New credit,...,0 days 00:00:00.007000,75,0 days 00:00:00.006000,0.01,0.01,75 days 14:18:05.754000,20 days 04:04:09.814735216,W_Call incomplete files,O_Cancelled,W_Validate application


In [10]:
df_2017.to_csv('BPI_2017_rfc_xgboost.csv')

In [11]:
# Save the best model to a file using pickle
model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model, model_file)

### XGBoost for time till next event prediction

In [12]:
df_2017 = pd.read_csv('BPI_2017_rfc_xgboost.csv')

# find out which columns we have to preprocess
num_columns = df_2017.select_dtypes(include=[np.number]).columns
num_imputer = SimpleImputer(strategy='median')
df_2017[num_columns] = num_imputer.fit_transform(df_2017[num_columns])

# Impute missing values for categorical columns
cat_columns = df_2017.select_dtypes(include=['object', 'category']).columns
cat_imputer = SimpleImputer(strategy='most_frequent', fill_value='missing')
df_2017[cat_columns] = cat_imputer.fit_transform(df_2017[cat_columns])

# Encoding categorical variables
label_encoders = {}
for column in cat_columns:
    le = LabelEncoder()
    df_2017[column] = le.fit_transform(df_2017[column])
    label_encoders[column] = le

# Calculate correlation for numerical features
correlations = df_2017[num_columns].corrwith(df_2017['time_to_next_event_seconds']).sort_values(ascending=False)
print("Correlations with the target:\n", correlations)

# UNCOMMENT ABOVE CODE  FOR MORE INFO ABOUT DATA

# # Prepare df_2017 for mutual information calculation
# # Drop the target variable and ensure no NaN values exist
# X = df_2017.drop(columns=['time_to_next_event_seconds'])
# y = df_2017['time_to_next_event_seconds']

# # Calculate mutual information
# mi_scores = mutual_info_regression(X, y, random_state=0)
# mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
# print("\nMutual Information Scores:\n", mi_scores)


/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_27900/3037175519.py:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2017 = pd.read_csv('BPI_2017_rfc_xgboost.csv')


Correlations with the target:
 time_to_next_event_seconds              1.000000
predicted_time_to_next_event_seconds    0.240850
case:RequestedAmount                    0.004244
NumberOfTerms                          -0.000543
MonthlyCost                            -0.000624
OfferedAmount                          -0.000823
FirstWithdrawalAmount                  -0.000990
CreditScore                            -0.001973
event_seq                              -0.004017
Unnamed: 0.1                           -0.013705
Unnamed: 0                             -0.013705
dtype: float64


In [13]:
X = df_2017.drop(['case:concept:name', 'Unnamed: 0','concept:name', 'EventOrigin',
       'EventID', 'case:LoanGoal', 'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID', 'predicted_time_to_next_event_seconds'], axis=1)  # Drop target and non-feature columns
y = df_2017['time_to_next_event_seconds']  # Target for prediction

# Split data into training, validation, and test sets (60%, 20%, 20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# timing
start_time = time.time()

# Preprocessing for numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ])

# Fit the preprocessor on the training data and transform both training and validation data
X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.transform(X_val)

# Initialize the model
model = XGBRegressor(objective='reg:squarederror')

# Fit the model using the transformed training set and validate using the transformed validation set
model.fit(X_train_transformed, y_train, eval_set=[(X_val_transformed, y_val)], early_stopping_rounds=10, verbose=False)

# Transform the test set and predict
X_test_transformed = preprocessor.transform(X_test)
y_pred_test = model.predict(X_test_transformed)

end_time = time.time()
# print the time for how long model runs
elapsed_time = end_time - start_time

print(f"Running time for model: {elapsed_time} seconds")

# Save the best model to a file using pickle
model_filename = 'xgboost_time.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model, model_file)

# Evaluate the model on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'Test RMSE: {rmse_test}')

r2_score_value = r2_score(y_test, y_pred_test)
print(f'R² score: {r2_score_value}')
# For predicting and adding new columns, you need to transform the whole dataset
X_transformed = preprocessor.transform(X)

df_2017['predicted_time_to_next_event_seconds_XG'] = np.abs(model.predict(X_transformed).round(1))

# decode labels
for column in label_encoders:
    print(column)
    le = label_encoders[column]
    print(le)
    df_2017[column] = le.inverse_transform(df_2017[column])

# drop unused columns
df_2017 = df_2017.drop(['elapsed_time_from_start', 'predicted_start_time', 'timestamp_difference', 'event_seq', 'Unnamed: 0.1', 'Unnamed: 0', 'time_to_next_event'], axis=1, errors='ignore')

# convert in proper timedeltas
df_2017['predicted_time_to_next_event_seconds_XG'] = df_2017['predicted_time_to_next_event_seconds_XG'].apply(lambda x: timedelta(seconds=x))
df_2017['time_to_next_event_seconds'] = df_2017['time_to_next_event_seconds'].apply(lambda x: timedelta(seconds=x))
df_2017['predicted_time_to_next_event_seconds'] = df_2017['predicted_time_to_next_event_seconds'].apply(lambda x: timedelta(seconds=x))

# return the naive model predictions on its place
df_2017['predicted_step'] = df_2017['predicted_step'].shift(-1)

# proper naming for columns
df_2017.rename(columns={'time_to_next_event_seconds': 'time_to_next_event'}, inplace=True, errors='ignore')
df_2017.rename(columns={'predicted_time_to_next_event_seconds': 'time_to_next_event_baseline'}, inplace=True,  errors='ignore')
df_2017.rename(columns={'predicted_time_to_next_event_seconds_XG': 'time_to_next_event_xgboost'}, inplace=True,  errors='ignore')
df_2017.rename(columns={'predicted_step': 'concept:name next naive'}, inplace=True,  errors='ignore')

/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/Process_Mining_1/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Running time for model: 1.7901618480682373 seconds
Test RMSE: 20920.835001461575
R² score: 0.9984493382777463
Action
LabelEncoder()
org:resource
LabelEncoder()
concept:name
LabelEncoder()
EventOrigin
LabelEncoder()
EventID
LabelEncoder()
lifecycle:transition
LabelEncoder()
time:timestamp
LabelEncoder()
case:LoanGoal
LabelEncoder()
case:ApplicationType
LabelEncoder()
case:concept:name
LabelEncoder()
Accepted
LabelEncoder()
Selected
LabelEncoder()
OfferID
LabelEncoder()
timestamp_difference
LabelEncoder()
time_to_next_event
LabelEncoder()
elapsed_time_from_start
LabelEncoder()
predicted_start_time
LabelEncoder()
predicted_step
LabelEncoder()
concept:name next
LabelEncoder()
concept:name next rfc
LabelEncoder()


In [14]:
df_2017

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,Selected,CreditScore,OfferedAmount,OfferID,time_to_next_event,time_to_next_event_baseline,concept:name next naive,concept:name next,concept:name next rfc,time_to_next_event_xgboost
0,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,New credit,Application_652823628,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.010000,0 days 00:00:35.170000,W_Complete application,W_Complete application,W_Complete application,0 days 00:00:48.900002
1,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,New credit,Application_652823628,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.010000,0 days 00:00:00.010000,A_Concept,A_Concept,A_Concept,0 days 00:00:48.900002
2,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413,Existing loan takeover,New credit,Application_652823628,...,True,0.0,15000.0,Offer_1000145087,4 days 23:34:08.080000,2 days 01:50:47.740000,A_Create Application,A_Create Application,A_Create Application,4 days 22:35:25
3,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500,Home improvement,New credit,Application_1691306052,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.050000,0 days 00:00:00.040000,A_Submitted,A_Submitted,A_Submitted,0 days 00:00:48.900002
4,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549,Home improvement,New credit,Application_1691306052,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.190000,0 days 00:00:00.300000,W_Handle leads,W_Handle leads,W_Handle leads,0 days 00:00:48.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201081,statechange,User_99,A_Pending,Application,ApplState_1437687321,complete,2017-02-01 09:58:43.970,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00,0 days 00:00:00,W_Call incomplete files,W_Validate application,W_Validate application,0 days 00:00:48.900002
1201082,Deleted,User_99,W_Validate application,Workflow,Workitem_1690791617,ate_abort,2017-02-01 09:58:43.972,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.010000,0 days 00:00:00.020000,W_Call incomplete files,O_Cancelled,W_Validate application,0 days 00:00:48.900002
1201083,statechange,User_99,O_Cancelled,Offer,OfferState_2080870299,complete,2017-02-01 09:58:43.981,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1237672415,0 days 00:00:00.010000,0 days 00:00:00.010000,W_Call incomplete files,O_Cancelled,W_Validate application,0 days 00:00:48.900002
1201084,statechange,User_99,O_Cancelled,Offer,OfferState_1140159487,complete,2017-02-01 09:58:43.988,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1471775802,0 days 00:00:00.010000,0 days 00:00:00.010000,W_Call incomplete files,O_Cancelled,W_Validate application,0 days 00:00:48.900002


In [15]:
df_2017.to_csv('BPI_2017_rfc_xgboost.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plotting feature importances
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

plt.title('Feature Importance', fontsize=20)
plt.xlabel('Importance', fontsize=18)
plt.ylabel('Feature', fontsize=18)

# Increase tick label size
barplot.xaxis.set_tick_params(labelsize=16)
barplot.yaxis.set_tick_params(labelsize=16)

plt.show()
plt.savefig('feature.png')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plotting feature importances
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

plt.title('Feature Importance', fontsize=20)
plt.xlabel('Importance', fontsize=18)
plt.ylabel('Feature', fontsize=18)

# Increase tick label size
barplot.xaxis.set_tick_params(labelsize=16)
barplot.yaxis.set_tick_params(labelsize=16)

plt.show()
plt.savefig('feature.png')

In [ ]:
predicted_counts = df_2017['concept:name next rfc'].value_counts()

plt.figure(figsize=(10, 7))
sns.barplot(x=predicted_counts.index, y=predicted_counts.values)
plt.title('Predicted Next Step Distribution')
plt.xlabel('Next Steps')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
class_names = label_encoder.inverse_transform(range(n_classes))

# Compute ROC curve and ROC area for each class
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting
plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of {class_names[i]} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve with True Class Names')
plt.legend(loc="lower right", prop={'size': 6})  # Adjust size as needed
plt.show()


In [ ]:
# Compute precision-recall curve and average precision for each class
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_binarized[:, i],
                                                        y_pred_proba[:, i])
    average_precision[i] = average_precision_score(y_test_binarized[:, i], y_pred_proba[:, i])

# Plotting
plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(recall[i], precision[i], label=f'{class_names[i]} (area = {average_precision[i]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve with True Class Names')
plt.legend(loc="best", prop={'size': 6})  # Adjust size as needed
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assume 'average_precision' is a dictionary containing the AP scores for each class
# where keys are numerical class indices and values are AP scores.

# Get class names from the label encoder
class_names = label_encoder.inverse_transform(sorted(average_precision.keys()))

# Get AP scores in the order of class names
ap_scores = [average_precision[i] for i in sorted(average_precision.keys())]

# Create a bar plot
plt.figure(figsize=(12, 8))
bars = plt.bar(range(len(class_names)), ap_scores, color='skyblue')

# Add the AP scores above the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, round(yval, 2), va='bottom')  # va: vertical alignment

plt.xlabel('Class Names')
plt.ylabel('Average Precision Score')
plt.title('Average Precision Scores by Class')
plt.xticks(range(len(class_names)), class_names, rotation=90)  # Rotate class names for better visibility
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Assuming 'y_test' are your true labels and 'y_pred' are your predicted labels
# If 'y_pred' is not yet defined, you would need to predict it using your model
y_pred = best_model.predict(X_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Assuming 'label_encoder' is the LabelEncoder instance used to encode your labels
# Use the inverse transform method to get the original class names
class_names = label_encoder.inverse_transform(np.arange(cm.shape[0]))

# Calculate error rates for each class
error_rate = (cm.sum(axis=1) - np.diag(cm)) / cm.sum(axis=1)

# Plot the error rate by class
plt.figure(figsize=(12, 8))
plt.bar(class_names, error_rate, color='salmon')
plt.xlabel('Class Names')
plt.ylabel('Error Rate')
plt.title('Error Rate by Class')
plt.xticks(rotation=90)  # Rotate the class names for better readability if needed
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Generate the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Convert the report to a DataFrame
report_df = pd.DataFrame(report).transpose()

# Replace index with class names for clarity, using label_encoder
class_names = label_encoder.inverse_transform(sorted(np.unique(y_test)))

# Exclude the 'accuracy' row and split the DataFrame into metrics and support
metrics_df = report_df.iloc[:-3, :-1]  # Exclude the 'support' column and the last 3 summary rows
support_df = report_df.iloc[:-3, -1:].astype(int)  # Keep only the 'support' column, convert to int
metrics_df.index = class_names
support_df.index = class_names

# Setup the matplotlib figure and axes for a side-by-side plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 10), gridspec_kw={'width_ratios': [3, 1]})

# Plot the heatmap for metrics
sns.heatmap(metrics_df, annot=True, fmt=".2f", cmap='Blues', ax=ax1, cbar=False, annot_kws={'size':18})
ax1.set_title('Precision, Recall, and F1-Score', fontsize=20)
ax1.set_ylabel('Classes', fontsize=18)
ax1.set_xlabel('Metrics', fontsize=18)
ax1.tick_params(axis='both', which='major', labelsize=16)

# Plot the heatmap (actually just annotated cells) for support
sns.heatmap(support_df, annot=True, fmt="d", cmap=None, ax=ax2, cbar=False, annot_kws={'size':18})
ax2.set_title('Support', fontsize=20)
ax2.set_xlabel('Metrics', fontsize=18)
ax2.tick_params(axis='y', which='major', labelsize=16)
ax2.set_yticks([]) 

plt.tight_layout()
plt.show()
plt.savefig('Precision_recall_F1_support_scores.jpg')

### Only for test purposes, do not run it

### Use this sample of code for future in Sprint 3

In [35]:
# create dataset
X = df_2017[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]
y = df_2017[['concept:name next']]

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
X['concept:name'] = label_encoder.fit_transform(X['concept:name'])
y = label_encoder.fit_transform(y)

# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=False, random_state=None)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # configure the cross-validation procedure
    cv_inner = KFold(n_splits=3, shuffle=False, random_state=None)
    # define the model
    model = RandomForestClassifier(n_jobs=1)
    # define search
    search = GridSearchCV(model, param_grid=param_grid, scoring='accuracy', cv=cv_inner, refit=True)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:9: SettingWithCop

>acc=0.980, est=0.950, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 19}
>acc=0.950, est=0.953, cfg={'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 19}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}
>acc=0.940, est=0.954, cfg={'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 21}
>acc=0.920, est=0.957, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 20}
>acc=0.919, est=0.955, cfg={'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimat

In [14]:
# Save the best model to a file
model_filename = 'random_forest_model.joblib'
joblib.dump(best_model, model_filename)

# Save the best model to a file using pickle
model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model, model_file)

In [18]:
df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,predicted_time_to_next_event_seconds_XG,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,concept:name next,predicted_next_event_rfc
3,3.0,1,0,22,2,701704,6,3,5,1,...,0 days 00:00:35.170000,33119,128,7,0 days 00:00:15,22.0,8.0,4.0,21.0,21
4,4.0,0,0,21,2,630431,3,4,5,1,...,0 days 00:00:00.010000,33126,127,6,0 days 00:00:05.500000,22.0,22.0,8.0,3.0,3
5,5.0,4,0,3,0,169679,1,5,5,1,...,2 days 01:50:47.740000,33133,126,0,4 days 22:54:39.812500,21.0,22.0,22.0,4.0,4
6,6.0,0,0,4,0,219156,1,6,7,1,...,0 days 00:00:00.040000,0,112,1,0 days 00:00:53.700001,3.0,21.0,22.0,8.0,8
7,7.0,4,0,8,0,130913,1,7,7,1,...,0 days 00:00:00.300000,49,113,3,0 days 00:00:11.200000,4.0,3.0,21.0,22.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201085,1201085.0,1,148,25,2,708416,0,1200970,11,1,...,0 days 00:00:00.020000,999899,44,5,0 days 00:01:00.799999,7.0,10.0,20.0,11.0,25
1201086,1201086.0,4,148,11,1,323681,1,1200971,11,1,...,0 days 00:00:00.010000,999900,45,5,0 days 00:01:17.500000,25.0,7.0,10.0,11.0,25
1201087,1201087.0,4,148,11,1,250417,1,1200972,11,1,...,0 days 00:00:00.010000,999901,43,5,0 days 00:01:17.500000,11.0,25.0,7.0,11.0,25
1201088,1201088.0,4,148,11,1,316489,1,1200973,11,1,...,0 days 00:00:00.010000,999902,47,5,0 days 00:01:17.500000,11.0,11.0,25.0,11.0,19
